In [ ]:
import numpy as np
import pandas as pd
import scipy.linalg as sla
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.linear_model import LinearRegression, Lasso, Ridge

**Регуляризация** -- это метод борьбы с переобучением, который штрафует модель за излишнюю сложность, что позволяет строить более простые (и потому стабильные) зависимости.

Еще одной проблемой, специфичной для линейных моделей, является **мультиколлинеарность**.Eсли оптимальных решений задачи минимизации оказывается бесконечно много, то коэффициенты модели могут принимать какие угодно огромные значения. **Вывод**: необходимо *ограничить величину* коэффициентов модели.

Для линейной модели дополнительные ограничения на веса выполняют роль регуляризации.

Функция потерь для линейной регрессии с регуляризацией выглядит следующим образом:
$$L(\mathbf{w}) = \frac{1}{\ell}\sum_{i=1}^{\ell}(\langle \mathbf{w} , x^i \rangle - y^i)^2 + R(\tilde{ \mathbf{w}}) \rightarrow \min_{w}$$
где $\tilde {\mathbf{w}} =  (w_1, \ldots, w_n)$ --- вектор весов без свободного члена $w_0$.

Последнее слагаемое определяет вид регуляризации.
* $L_1$-регуляризация (LASSO, least absolute shrinkage and selection operator):
$$R( \mathbf{\tilde w}) = \lambda|| \mathbf{\tilde {w}}||_1 = \lambda (|w_1| + \ldots + |w_n|);$$
* $L_2$-регуляризация (Ridge):
$$R(\tilde{ \mathbf{w}}) = \lambda||\tilde{ \mathbf{w}}||_2^2 = \lambda(|w_1|^2 + \ldots + |w_n|^2);$$
* ElasticNet -- комбинация двух предыдущих:
 $$R(\tilde{ \mathbf{w}}) = \alpha ||\tilde{\mathbf{w}}||_1+ \beta ||\tilde {\mathbf{w}}||_2^2.$$



## Lasso-регрессия

В LASSO-регрессии штрафуем модель **на сумму модулей всех ее весов**.

**Лосс:** $$L(\mathbf{w}) = \frac{1}{\ell} ||X\mathbf{w} - \mathbf{y}||^2_2 + \lambda ||\tilde{\mathbf{w}}||_1,$$ где $\lambda$ -- гиперпараметр, отвечающий за степень регуляризации.

В привычном понимании:

**Лосс:** $$L(\mathbf{w}) = \frac{1}{\ell}\sum_{i=1}^{\ell}\left(\sum_{j=0}^{n} x_{ij}w_j - y_i\right)^2 + \lambda\sum_{j=1}^{n}|w_j|$$



**Градиент:**
$$
\frac{\partial{L}}{\partial{\mathbf{w}}}
= \frac{2}{\ell}\cdot X^T(X\mathbf{w} - \mathbf{y}) + \lambda (0, \mathrm{sign}(w_1), \ldots, \mathrm{sign}(w_n))^T.
$$

In [ ]:
def soft_sign(x, eps=1e-7):
    if abs(x) > eps:
        return np.sign(x)
    return x / eps

np_soft_sign = np.vectorize(soft_sign)


class MyLassoRegression(object):
    def __init__(self, C=1):
        self.coef_ = None
        self.intercept_ = None
        self.C = C

    def regularization_term(self, weights):
        signs =
        signs[0] = 0 
        return signs

    def grad(self, X, y, weights):
        y_pred = (X @ weights)  # [l, 1]

        basic_term = 

        regularization_term = self.regularization_term(weights)

        return basic_term + self.C * regularization_term 


    def fit(self, X, y, max_iter=100, lr=0.1):
        X = np.array(X)
        y = np.array(y)
        assert len(y.shape) == 1 and len(X.shape) == 2
        assert X.shape[0] == y.shape[0]

        y = y[:, np.newaxis]

        l, n = X.shape
        X_train = np.hstack([np.ones([l, 1]), X])  # [ell, n + 1]

        weights = np.random.randn(n + 1, 1)

        losses = []

        for iter_num in range(max_iter):
            grad = self.grad(X_train, y, weights)
            # update weights
            weights -= grad * lr / ((iter_num + 1) ** 0.5)

            loss = np.mean((X_train @ weights - y) ** 2) + self.C * np.sum(np.abs(weights[1:]))
            losses.append(loss)

        self.coef_ = weights[1:]
        self.intercept_ = weights[0]

        return losses


    def predict(self, X):
        X = np.array(X)
        y_pred = X @ self.coef_ + self.intercept_

        return y_pred